# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install lxml
!pip install et_xmlfile
!pip install html5lib
!pip install bs4

import urllib.request
from bs4 import BeautifulSoup

! pip install folium==0.5.0
import folium

print('Libraries imported.')

Libraries imported.


In [2]:
#try to fucking scrape a wikipedia page with Beautiful Soup
wiki = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(wiki, "lxml")

In [3]:
#Populating HTML into a dataframe 
right_table=soup.find('table', class_='wikitable sortable')

A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#dropping Neighborhoods that are 'Not assigned'
df = df[~df.Borough.str.contains("Not assigned")]
df.sort_values(by=['Postal Code'], inplace=True)
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
#show the number of rows of cleaned data set
print("The size of the cleaned data frame is ",df.shape)

The size of the cleaned data frame is  (103, 3)


In [6]:
#adding Lat Long
latlong = pd.read_csv("https://cocl.us/Geospatial_data")
latlong.sort_values(by=['Postal Code'], inplace=True)
latlong = latlong.reset_index(drop=True)
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df1 = df.merge(latlong, how='left', left_index=True, right_index=True)
df1 = df1.reset_index(drop=True)
df1 = df1.rename(columns={"Postal Code_x": "Postal Code"})
df1 = df1.drop(["Postal Code_y"], axis=1)
df1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
